In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!wget "http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat"

!wget "http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat"

In [ ]:
!ls

In [ ]:
from scipy.io import loadmat

In [ ]:
## Read the data.
dataset = loadmat('Indian_pines_corrected.mat')['indian_pines_corrected']
ground_truth = loadmat('Indian_pines_gt.mat')['indian_pines_gt']

In [ ]:
#The dimensions of the data.
print(f'Dataset: {dataset.shape}\nGround Truth: {ground_truth.shape}')

In [ ]:
import numpy as np #define f
import matplotlib.pyplot as plt

def plot_band(dataset):
    plt.figure(figsize=(8, 6))
    band_no = np.random.randint(dataset.shape[2])
    plt.imshow(dataset[:,:, band_no], cmap='jet')
    plt.title(f'Band-{band_no}', fontsize=14)
    plt.axis('off')
    plt.colorbar()
    plt.show()

In [ ]:
plot_band(dataset)

In [ ]:
plt.figure(figsize=(8, 6)) #pi
plt.imshow(ground_truth, cmap='jet')
plt.axis('off')
plt.colorbar(ticks= range(0,16))
plt.show()

In [ ]:
from tqdm import tqdm

In [ ]:
dataset[:,:,2].ravel().shape

In [ ]:
ser = pd.Series(dataset[0][0])

In [ ]:
ser

In [ ]:
df_temp = pd.DataFrame()
temp = pd.Series([1,3])
df_temp = pd.concat([temp,ser])

In [ ]:
df_temp

In [ ]:
pd.DataFrame(dataset[0,0])

In [ ]:
df_new = pd.DataFrame()
for i in tqdm(range(dataset.shape[0])):
    for j in range(dataset.shape[1]):
        df_new=pd.concat([df_new,pd.Series([i,j]),pd.Series(dataset[i,j])],axis=0)

In [ ]:
row_list = []
for i in range(0,145):
    for j in range(0,145):
        row_list.append(i)

col_list = []
for i in range(0,145):
    for j in range(0,145):
        col_list.append(j)

row_series = pd.Series(row_list)
col_series = pd.Series(col_list)

In [ ]:
row_series

In [ ]:
#index_ser.shape

In [ ]:
dataset[:,:,0].shape

In [ ]:
from tqdm import tqdm
import pandas as pd
def extract_pixels(dataset, ground_truth):
    df = pd.DataFrame()
    df = pd.concat([df,row_series,col_series],axis=1)
    for i in tqdm(range(dataset.shape[2])):
        df = pd.concat([df, pd.DataFrame(dataset[:, :, i].ravel())], axis=1)
    df = pd.concat([df, pd.DataFrame(ground_truth.ravel())], axis=1)
    df.columns = ['pixel_row', 'pixel_col']+[f'band-{i}' for i in range(1, 1+dataset.shape[2])]+['class']
    return df

In [ ]:
df = extract_pixels(dataset, ground_truth)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv('Dataset.csv', index=False)

In [ ]:
df.iloc[:,2]

In [ ]:
def plot_signature(df):
    plt.figure(figsize=(12, 6))
    pixel_no = np.random.randint(df.shape[0])
    plt.plot(range(1, 201), df.iloc[pixel_no, 2:-1].values.tolist(), 'b--', label= f'Class - {df.iloc[pixel_no, -1]}')
    plt.legend()
    plt.title(f'Pixel({pixel_no}) signature', fontsize=14)
    plt.xlabel('Band Number', fontsize=14)
    plt.ylabel('Pixel Intensity', fontsize=14)
    plt.show()

plot_signature(df)

In [ ]:
df.loc[:,'class'].unique()

In [ ]:
df.loc[:, 'class'].value_counts()

In [ ]:
import seaborn as sns

plt.figure(figsize=(14, 8))

ax = sns.countplot(x='class', data=df[['class']])

for p in ax.patches:
        ax.annotate('{:.1f}%'.format(100*p.get_height()/df.shape[0]), (p.get_x()+0.1, p.get_height()+5))

plt.ylabel('Class count with percentage', fontsize=14)

plt.xlabel('class', fontsize=14)

plt.title('Bar Plot', fontsize=16)

plt.show()

In [ ]:
# Visualizing the pixel
plt.matshow(df.iloc[1, 2:-1].values.reshape(1,200), cmap='jet');

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
def plot_pixel(pixel, n):
    ax = plt.gca()
    im = ax.imshow(pixel.reshape(10, 20), cmap = 'jet')
    plt.title(f'pixel - {n}')
    plt.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
plt.figure(figsize=(8, 4))

rand_n = np.random.randint(df.shape[0])

plot_pixel(df.iloc[rand_n, 2:-1].values, rand_n)

In [ ]:
type(df['band-1'])

In [ ]:
plt.figure(figsize=(16, 6))
n = int(input('Enter the band Number(1-200) :'))
sns.boxplot( x=df["class"], y=df[str('band-'+str(n))], width=0.3);
plt.title('Box Plot', fontsize= 16)
plt.xlabel('Class', fontsize= 14)
plt.ylabel(f'Band-{n}', fontsize= 14)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('Dataset.csv')

In [ ]:
df.head()

In [ ]:
X = df.iloc[:, 2:-1].values

y = df.iloc[:, -1].values

In [ ]:
X.shape,y.shape

In [ ]:
pca = PCA(n_components=30)
principalComponents = pca.fit_transform(X)
ev = pca.explained_variance_ratio_

In [ ]:
ev

In [ ]:
plt.plot(np.cumsum(ev))
plt.xlabel("Number of components")
plt.ylabel("cumulative explained variance")
plt.show()

In [ ]:
# Interactive plot
df3 = pd.DataFrame(ev, columns=['Cumulative explained variance']).cumsum()
df3['Number of components'] = pd.Series(list(range(30)))
df3.plot(x='Number of components', y='Cumulative explained variance')

In [ ]:
X_train, X_test, y_train, y_test, indices_train, indices_test  = train_test_split(principalComponents, y,  range(X.shape[0]),
                                                                                  test_size = 0.15)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
svm = SVC(kernel='rbf', degree = 10, gamma='scale', cache_size=1024*7)
svm.fit(X_train, y_train)

In [ ]:
y_pred = svm.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Accuracy: ", accuracy_score(y_test, y_pred))

In [ ]:
accuracy_list = []
for i in range(0,20):
    X_train, X_test, y_train, y_test, indices_train, indices_test  = train_test_split(principalComponents, y,  range(X.shape[0]),
                                                                                  test_size = 0.15)
    svm = SVC(kernel='rbf', degree = 10, gamma='scale', cache_size=1024*7)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    accuracy_list.append(accuracy_score(y_test, y_pred))
    #print(f"Accuracy : {accuracy_score(y_test,y_pred)}")

print(f"Average accuracy: {sum(accuracy_list)/len(accuracy_list)}")

In [ ]:
!pip install scikit-plot


In [ ]:
import scikitplot as skplt

skplt.metrics.plot_confusion_matrix(
    y_test,
    y_pred,
    figsize=(12,12))

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   z= confusion_matrix(y_test, y_pred),
                   x=[f'class-{i}' for i in np.unique(df.loc[:, 'class'].values)],
                   y=[f'class-{i}' for i in np.unique(df.loc[:, 'class'].values)],
                   hoverongaps = False))
fig.show()

In [ ]:
print('Classification report:\n',classification_report(y_test,y_pred))

In [ ]:
df.iloc[:,-1].values.reshape((145,145))

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(df.iloc[:, -1].values.reshape((145, 145)), cmap='jet')
plt.colorbar()
plt.axis('off')
plt.title('Ground Truth')
plt.savefig('ground_truth.png')
plt.show()

In [ ]:
len([0]*X.shape[0])

In [ ]:
pre = y_pred
clmap = np.zeros((145,145))

for i in tqdm(range(len(indices_train))):
    clmap[df.iloc[indices_train[i], 0]][df.iloc[indices_train[i], 1]] = y[indices_train[i]]

for i in tqdm(range(len(indices_test))):
    clmap[df.iloc[indices_test[i], 0]][df.iloc[indices_test[i], 1]] = pre[i]

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(clmap, cmap='jet')
plt.colorbar()
plt.axis('off')
plt.title('Classification Map (PCA + SVM)')
plt.savefig('Classification_map.png')
plt.show()

In [ ]:
from tqdm import tqdm
pre = y_pred

clmap = [0]*X.shape[0]

for i in tqdm(range(len(indices_train))):
    clmap[indices_train[i]] = y[indices_train[i]]

for i in tqdm(range(len(indices_test))):
    clmap[indices_test[i]] = pre[i]

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(np.array(clmap).reshape((145, 145)), cmap='jet')
plt.colorbar()
plt.axis('off')
plt.title('Classification Map (NN)')
plt.savefig('Classification_map.png')
plt.show()

NN


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers, models, layers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
X_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras import regularizers, models, layers
from keras.callbacks import EarlyStopping


model = Sequential()

model.add(Dense(150,activation='relu',kernel_regularizer=regularizers.l2(1e-4)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(100,activation='relu',kernel_regularizer=regularizers.l2(1e-4)))
model.add(BatchNormalization())

model.add(Dense(100,activation='relu',kernel_regularizer=regularizers.l2(1e-4)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(75,activation='relu',kernel_regularizer=regularizers.l2(1e-4)))
model.add(BatchNormalization())

model.add(Dense(75,activation='relu',kernel_regularizer=regularizers.l2(1e-4)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(50,activation='relu',kernel_regularizer=regularizers.l2(1e-4)))
model.add(BatchNormalization())

model.add(Dense(17,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
batch_size = 128
epochs = 200
early_stop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=20)
model.fit(x=X_train,
          y=y_train,
          epochs=epochs,
          validation_data=(X_test,y_test),
          verbose=0,
         callbacks=[early_stop])

In [ ]:
model.summary()

In [ ]:
score = model.evaluate(X_test,y_test, verbose=1)

In [ ]:
score

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_classes = y_pred.argmax(axis=1)

In [ ]:
y_pred_classes

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Accuracy: ", accuracy_score(y_test, y_pred_classes))

In [ ]:
import scikitplot as skplt

skplt.metrics.plot_confusion_matrix(
    y_test,
    y_pred_classes,
    figsize=(12,12))

In [ ]:
print('Classification report:\n',classification_report(y_test,y_pred_classes))

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(df.iloc[:, -1].values.reshape((145, 145)), cmap='jet')
plt.colorbar()
plt.axis('off')
plt.title('Ground Truth')
plt.savefig('ground_truth.png')
plt.show()

In [ ]:
from tqdm import tqdm
pre = y_pred_classes

clmap = [0]*X.shape[0]

for i in tqdm(range(len(indices_train))):
    clmap[indices_train[i]] = y[indices_train[i]]

for i in tqdm(range(len(indices_test))):
    clmap[indices_test[i]] = pre[i]

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(np.array(clmap).reshape((145, 145)), cmap='jet')
plt.colorbar()
plt.axis('off')
plt.title('Classification Map (NN)')
plt.savefig('Classification_map.png')
plt.show()